# Scrapper PORTALCHILE.ORG

## Data

### load the data 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('assets/datos_empresas.csv', index_col='DONANTE')

df.head()

,name,item,subitem,activity,rut
DONANTE,,,,,
DEPOCARGO DEPÃSITO ADUANERO DE CARGA LTDA.,Depocargo Deposito Aduanero De Carga Limitada,"J - Transporte, Almacenamiento Y Comunicaciones",630 - Act. De Transporte Complementarias Y Au...,"630100 - Manipulacion De La Carga,630200 - Ser...",96.888.200-7
LIMITADA OLIVARES OLIVARES,Olivares Olivares Limitada,D - Industrias Manufactureras No Metalicas,222 - Actividades De Impresion Y De Servicios ...,222200 - Actividades De Servicio Relacionada C...,82.437.600-K
CHARTERING Y TRADING JAVIER,Chartering Y Trading Javier Bade Eirl,"J - Transporte, Almacenamiento Y Comunicacione...",602 - Otros Tipos De Transporte Por Via Terre...,"602300 - Transporte De Carga Por Carretera,611...",76.838.470-3
RENCORET Y RIUTTIMANN ARQUITECTURA E INVERSIONES LTDA,Rencoret Y Ruttimann Arquitectura E Inversione...,"L - Actividades Inmobiliarias, Empresariales Y...",742 - Actividades De Arquitectura E Ingenieri...,742110 - Servicios De Arquitectura Y Tecnico R...,78.265.910-3
ASESORIAS E INVERSIONES AHUMADA LTDA.,Asesorias E Inversiones Ahumada Limitada,"A - Agricultura, Ganaderia, Caza Y Silvicultur...",014 - Actividades De Servicios Agricolas Y Gan...,"014019 - Otros Servicios Agricolas N.c.p.,6599...",76.092.180-7


### save the name used to make the query

In [3]:
names = df.name.str.lower().str.replace(' ','-')
ruts =  df.rut.str.replace('.','', regex=True)
ruts = ruts.str.split('-', expand=True)[0]
webname = names + '-' + ruts
df = df.assign(webname = webname)

## use soup

In [4]:
from bs4 import BeautifulSoup
import requests

### helper functions

In [5]:

def get_atts(soup):

    atts = {}
    
    div = soup.find(class_ = 'profile-header')
    p = div.find('p', class_ = 'thin')
    atts['Rubro'] =   p.find('a').text

    div_list = soup.find_all(class_ = 'profile-followers')
    for div in div_list:
        a_list = div.find_all('a')
        for a in a_list:
            att_name = a.find('em').text
            atts[att_name] = a.text.replace(att_name,'')
    return atts

### make da requests

In [6]:
url_template = 'https://www.portalchile.org/empresa/'

In [7]:

dict_df = {}
for i in webname.index:

    html_text = requests.get(url_template + webname[i]).text
    soup = BeautifulSoup(html_text, 'lxml')

    try:
        dict_df[i] = get_atts(soup)
    except AttributeError:
        print(i)
    


ISABEL RAMM CAMBIASO
ALEXANDER LOWENSTEIN VASQUEZ
MONICA DEL ROSARIO PEREZ OBREQUE
SOCIEDAD COMERCIALIZADORA PROMATHER S. A.
DISTRIBUIDORA DE ARTICULOS MEDICOS
DISTRIBUIDORA DE GAS VEGA LTDA.
SIMONETTI INMOVILIARIA S.A.
SQM SALAR S.A.
SOCIEDAD COMERCIAL SAN ANTONIO Y CIA. LTDA.
JORGE VALENZUELA
INVERSIONES DESARROLLO  SPA.
COMERCIAL G Y G LTDA.
GESTION AMBIENTAL S.A
NEA CONSULTORES ASOCIADOS LIMITADA
CLAUDIO RIVERA Y COMPANIA
INVERSIONES JACARANDA LTDA
SOLUCIONES BIOTECNOLOGICAS SPA
SOC. DE INVERSIONES Y ASESORIAS SANTA CATALINA LTDA
CGS CHILE LTDA
JORGE MERINO MORALES
RODRIGO GALLARDO DIAZ
FUND ANGELINI
SERVICIOS PROFESIONALES CARRASCO GALDAMES Y CIA
INMOBILIARIA PLAYA MANSA S.A.
SINDICATO DE EMPRESA DE TRANSPORTE FERROVIARIO S A TRACCION N 4
PROYECTOS Y CONSTRUCCIONES MADRID SPA
CLAUDIO MUÃOZ
BRUNO GATTINI KALWITZ
INGENIERIA CGSA
INVERSIONES SAN BENITO LTDA
COMERCIAL SANTIAGO LTDA
INVERSIONES AMAL LTDA
COMERCIALIZADORA DE PRODUCTOS METALÃRGICOS S.A
SIND PESCADORES ARTESA INDEP DE V

In [8]:
new_df = pd.DataFrame.from_dict(dict_df, orient='index') 
print(len(df))
new_df = new_df.merge(df, left_index=True, right_index=True)
print(len(new_df))

517
482


In [9]:
new_df.to_csv('outs/datos_buiss.csv')